In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd

In [4]:
from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.loss_utils import CSCELoss, create_ordinal_cost_matrix
from ordinaloss.engine.model_engine import OrdinalEngine


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models


In [ ]:
print("MM")

In [ ]:
print(2)

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
test_transform = create_transform_pipeline() #Without color jitter for test pipeline

train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

In [ ]:
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

print("asd!")

In [ ]:
def my_lambda_scheduler(epoch):
    return (0.8 ** (epoch // 5))

engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)

In [ ]:
engine.train(train_loader, test_loader, n_epochs = 15)